<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/AA-Stagecoach-Project/blob/main/File_share_46_Arrival_predictions_corrected_with_Headway_calculation_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

15/07/2024

In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz
import time

def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if len(data) == 0:
            return pd.DataFrame(), None  # No data available
        station_name = data[0]['stationName']
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')
            })
        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (BST)'] = pd.to_datetime(df['Expected Arrival (BST)'])  # Convert to datetime
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Gap'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['2_Gap'] = (df['Gap'] * 2).round(2)
        df['Gap_Sq'] = (df['Gap'] * df['Gap']).round(2)
        return df, station_name
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None

def main():
    line_id = input("Enter Line ID: ")
    stop_point_id = input("Enter Stop Point ID: ")
    direction = input("Enter Direction (inbound/outbound): ")

    cumulative_df = pd.DataFrame(columns=[
        'Line', 'Vehicle ID', 'Stop Point', 'Direction',
        'Expected Arrival (BST)', 'Expected Arrival (HM)',
        'Gap', '2_Gap', 'Gap_Sq'
    ])

    while True:
        arrival_predictions_df, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)

        if arrival_predictions_df is not None and not arrival_predictions_df.empty:
            current_hour = datetime.now(pytz.timezone('Europe/London')).hour
            next_hour = (current_hour + 1) % 24

            for _, row in arrival_predictions_df.iterrows():
                vehicle_id = row['Vehicle ID']
                expected_hour = row['Expected Arrival (BST)'].hour
                if (vehicle_id in cumulative_df['Vehicle ID'].values and
                    expected_hour in [current_hour, next_hour]):
                    cumulative_df.loc[(cumulative_df['Vehicle ID'] == vehicle_id) &
                                      (cumulative_df['Expected Arrival (BST)'].dt.hour.isin([current_hour, next_hour])), :] = row.values
                else:
                    cumulative_df = pd.concat([cumulative_df, row.to_frame().T]).reset_index(drop=True)

            # Recalculate headway for cumulative_df
            cumulative_df = cumulative_df.sort_values(by='Expected Arrival (BST)', ascending=True)
            cumulative_df['Expected Arrival (BST)'] = pd.to_datetime(cumulative_df['Expected Arrival (BST)'])  # Ensure datetime format
            cumulative_df['Expected Arrival (HM)'] = pd.to_datetime(cumulative_df['Expected Arrival (HM)'], format='%H:%M')

            # Identify the first imp_row
            cumulative_df['Gap'] = cumulative_df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
            cumulative_df['Gap'] = cumulative_df.apply(
                lambda row: 0 if row.name == 0 or row['Expected Arrival (BST)'].hour != cumulative_df.loc[row.name - 1, 'Expected Arrival (BST)'].hour else row['Gap'],
                axis=1
            )

            cumulative_df['2_Gap'] = (cumulative_df['Gap'] * 2).round(2)
            cumulative_df['Gap_Sq'] = (cumulative_df['Gap'] * cumulative_df['Gap']).round(2)

            # Calculate the number of buses observed in the current hour
            num_buses_observed = cumulative_df[cumulative_df['Expected Arrival (BST)'].dt.hour == current_hour].shape[0]

            print(f"\nArrival Predictions for stop point {stop_point_id} ({station_name}):")
            print(arrival_predictions_df.to_string(index=False))
            print("\nCumulative DataFrame:")
            print(cumulative_df.to_string(index=False))
            print(f"\nNumber of buses observed in the current hour: {num_buses_observed}")
        else:
            print("No arrival predictions available.")

        print("Refreshing data in 30 seconds...\n")
        time.sleep(30)  # Wait for 30 seconds before fetching data again

if __name__ == "__main__":
    main()


Enter Line ID: 141
Enter Stop Point ID: 490015195M
Enter Direction (inbound/outbound): outbound

Arrival Predictions for stop point 490015195M (City Road / Leonard Street):
Line Vehicle ID Stop Point Direction Expected Arrival (BST) Expected Arrival (HM)  Gap  2_Gap  Gap_Sq
 141    LK66GFE 490015195M  outbound    2024-07-16 16:16:11   1900-01-01 16:16:00  0.0    0.0     0.0
 141    LC67AHJ 490015195M  outbound    2024-07-16 16:27:50   1900-01-01 16:27:00 11.0   22.0   121.0

Cumulative DataFrame:
Line Vehicle ID Stop Point Direction Expected Arrival (BST) Expected Arrival (HM)  Gap  2_Gap  Gap_Sq
 141    LK66GFE 490015195M  outbound    2024-07-16 16:16:11   1900-01-01 16:16:00  0.0    0.0     0.0
 141    LC67AHJ 490015195M  outbound    2024-07-16 16:27:50   1900-01-01 16:27:00 11.0   22.0   121.0

Number of buses observed in the current hour: 0
Refreshing data in 30 seconds...


Arrival Predictions for stop point 490015195M (City Road / Leonard Street):
Line Vehicle ID Stop Point Direc

KeyboardInterrupt: 

16/07/2024

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz
import time

def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if len(data) == 0:
            return pd.DataFrame(), None  # No data available
        station_name = data[0]['stationName']
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')
            })
        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (BST)'] = pd.to_datetime(df['Expected Arrival (BST)'])  # Convert to datetime
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Gap'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['2_Gap'] = (df['Gap'] * 2).round(2)
        df['Gap_Sq'] = (df['Gap'] * df['Gap']).round(2)
        return df, station_name
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None

def main():
    line_id = input("Enter Line ID: ")
    stop_point_id = input("Enter Stop Point ID: ")
    direction = input("Enter Direction (inbound/outbound): ")

    cumulative_df = pd.DataFrame(columns=[
        'Line', 'Vehicle ID', 'Stop Point', 'Direction',
        'Expected Arrival (BST)', 'Expected Arrival (HM)',
        'Gap', '2_Gap', 'Gap_Sq'
    ])

    while True:
        arrival_predictions_df, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)

        if arrival_predictions_df is not None and not arrival_predictions_df.empty:
            current_hour = datetime.now(pytz.timezone('Europe/London')).hour
            next_hour = (current_hour + 1) % 24

            for _, row in arrival_predictions_df.iterrows():
                vehicle_id = row['Vehicle ID']
                expected_hour = row['Expected Arrival (BST)'].hour
                if (vehicle_id in cumulative_df['Vehicle ID'].values and
                    expected_hour in [current_hour, next_hour]):
                    cumulative_df.loc[(cumulative_df['Vehicle ID'] == vehicle_id) &
                                      (cumulative_df['Expected Arrival (BST)'].dt.hour.isin([current_hour, next_hour])), :] = row.values
                else:
                    cumulative_df = pd.concat([cumulative_df, row.to_frame().T]).reset_index(drop=True)

            # Recalculate headway for cumulative_df
            cumulative_df = cumulative_df.sort_values(by='Expected Arrival (BST)', ascending=True)
            cumulative_df['Expected Arrival (BST)'] = pd.to_datetime(cumulative_df['Expected Arrival (BST)'])  # Ensure datetime format
            cumulative_df['Expected Arrival (HM)'] = pd.to_datetime(cumulative_df['Expected Arrival (HM)'], format='%H:%M')

            # Identify the first imp_row
            cumulative_df['Gap'] = cumulative_df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
            cumulative_df['Gap'] = cumulative_df.apply(
                lambda row: 0 if row.name == 0 or row['Expected Arrival (BST)'].hour != cumulative_df.loc[row.name - 1, 'Expected Arrival (BST)'].hour else row['Gap'],
                axis=1
            )

            cumulative_df['2_Gap'] = (cumulative_df['Gap'] * 2).round(2)
            cumulative_df['Gap_Sq'] = (cumulative_df['Gap'] * cumulative_df['Gap']).round(2)

            # Calculate the number of buses observed in the current hour
            num_buses_observed = cumulative_df[cumulative_df['Expected Arrival (BST)'].dt.hour == current_hour].shape[0]

            print(f"\nArrival Predictions for stop point {stop_point_id} ({station_name}):")
            print(arrival_predictions_df.to_string(index=False))
            print("\nCumulative DataFrame:")
            print(cumulative_df.to_string(index=False))
            print(f"\nNumber of buses observed in the current hour: {num_buses_observed}")
        else:
            print("No arrival predictions available.")

        print("Refreshing data in 30 seconds...\n")
        time.sleep(30)  # Wait for 30 seconds before fetching data again

if __name__ == "__main__":
    main()


Enter Line ID: 141
Enter Stop Point ID: 490015195M
Enter Direction (inbound/outbound): outbound

Arrival Predictions for stop point 490015195M (City Road / Leonard Street):
Line Vehicle ID Stop Point Direction Expected Arrival (BST) Expected Arrival (HM)  Gap  2_Gap  Gap_Sq
 141    LK66GFE 490015195M  outbound    2024-07-16 16:16:11   1900-01-01 16:16:00  0.0    0.0     0.0
 141    LC67AHJ 490015195M  outbound    2024-07-16 16:27:50   1900-01-01 16:27:00 11.0   22.0   121.0

Cumulative DataFrame:
Line Vehicle ID Stop Point Direction Expected Arrival (BST) Expected Arrival (HM)  Gap  2_Gap  Gap_Sq
 141    LK66GFE 490015195M  outbound    2024-07-16 16:16:11   1900-01-01 16:16:00  0.0    0.0     0.0
 141    LC67AHJ 490015195M  outbound    2024-07-16 16:27:50   1900-01-01 16:27:00 11.0   22.0   121.0

Number of buses observed in the current hour: 0
Refreshing data in 30 seconds...


Arrival Predictions for stop point 490015195M (City Road / Leonard Street):
Line Vehicle ID Stop Point Direc

KeyboardInterrupt: 

16/07/2024-16.42

In [10]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz
import time

def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if len(data) == 0:
            return pd.DataFrame(), None  # No data available
        station_name = data[0]['stationName']
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')
            })
        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (BST)'] = pd.to_datetime(df['Expected Arrival (BST)'])  # Convert to datetime
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Gap'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['2_Gap'] = (df['Gap'] * 2).round(2)
        df['Gap_Sq'] = (df['Gap'] * df['Gap']).round(2)
        return df, station_name
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None

def main():
    line_id = input("Enter Line ID: ")
    stop_point_id = input("Enter Stop Point ID: ")
    direction = input("Enter Direction (inbound/outbound): ")

    cumulative_df = pd.DataFrame(columns=[
        'Line', 'Vehicle ID', 'Stop Point', 'Direction',
        'Expected Arrival (BST)', 'Expected Arrival (HM)',
        'Gap', '2_Gap', 'Gap_Sq'
    ])

    while True:
        arrival_predictions_df, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)

        if arrival_predictions_df is not None and not arrival_predictions_df.empty:
            current_hour = datetime.now(pytz.timezone('Europe/London')).hour

            for _, row in arrival_predictions_df.iterrows():
                vehicle_id = row['Vehicle ID']
                if (vehicle_id in cumulative_df['Vehicle ID'].values):
                    cumulative_df.loc[(cumulative_df['Vehicle ID'] == vehicle_id), :] = row.values
                else:
                    cumulative_df = pd.concat([cumulative_df, row.to_frame().T]).reset_index(drop=True)

            # Recalculate headway for cumulative_df
            cumulative_df = cumulative_df.sort_values(by='Expected Arrival (BST)', ascending=True)
            cumulative_df['Expected Arrival (BST)'] = pd.to_datetime(cumulative_df['Expected Arrival (BST)'])  # Ensure datetime format
            cumulative_df['Expected Arrival (HM)'] = pd.to_datetime(cumulative_df['Expected Arrival (HM)'], format='%H:%M')

            # Calculate the difference in minutes between consecutive arrivals
            cumulative_df['Gap'] = cumulative_df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60

            cumulative_df['2_Gap'] = (cumulative_df['Gap'] * 2).round(2)
            cumulative_df['Gap_Sq'] = (cumulative_df['Gap'] * cumulative_df['Gap']).round(2)

            # Calculate the number of buses observed in the current hour
            num_buses_observed = cumulative_df[cumulative_df['Expected Arrival (BST)'].dt.hour == current_hour].shape[0]

            print(f"\nArrival Predictions for stop point {stop_point_id} ({station_name}):")
            print(arrival_predictions_df.to_string(index=False))
            print("\nCumulative DataFrame:")
            print(cumulative_df.to_string(index=False))
            print(f"\nNumber of buses observed in the current hour: {num_buses_observed}")
        else:
            print("No arrival predictions available.")

        print("Refreshing data in 30 seconds...\n")
        time.sleep(30)  # Wait for 30 seconds before fetching data again

if __name__ == "__main__":
    main()


Enter Line ID: 141
Enter Stop Point ID: 490015195M
Enter Direction (inbound/outbound): outbound

Arrival Predictions for stop point 490015195M (City Road / Leonard Street):
Line Vehicle ID Stop Point Direction Expected Arrival (BST) Expected Arrival (HM)  Gap  2_Gap  Gap_Sq
 141    BU14EFZ 490015195M  outbound    2024-07-16 16:44:23   1900-01-01 16:44:00  0.0    0.0     0.0
 141    LK66GFX 490015195M  outbound    2024-07-16 16:50:28   1900-01-01 16:50:00  6.0   12.0    36.0
 141    LK66GFA 490015195M  outbound    2024-07-16 16:58:57   1900-01-01 16:58:00  8.0   16.0    64.0
 141    LC67AHE 490015195M  outbound    2024-07-16 17:07:47   1900-01-01 17:07:00  9.0   18.0    81.0

Cumulative DataFrame:
Line Vehicle ID Stop Point Direction Expected Arrival (BST) Expected Arrival (HM)  Gap  2_Gap  Gap_Sq
 141    BU14EFZ 490015195M  outbound    2024-07-16 16:44:23   1900-01-01 16:44:00  0.0    0.0     0.0
 141    LK66GFX 490015195M  outbound    2024-07-16 16:50:28   1900-01-01 16:50:00  6.0   1

KeyboardInterrupt: 

16/07/24 File share 46- Arrival predictions corrected with Headway calculation correction

In [12]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz
import time

def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if len(data) == 0:
            return pd.DataFrame(), None, pd.DataFrame(), 0  # No data available

        station_name = data[0]['stationName']
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')
            })
        arrival_predictions_df = pd.DataFrame(predictions)
        arrival_predictions_df = arrival_predictions_df.sort_values(by='Expected Arrival (BST)', ascending=True)
        arrival_predictions_df['Expected Arrival (BST)'] = pd.to_datetime(arrival_predictions_df['Expected Arrival (BST)'])  # Convert to datetime
        arrival_predictions_df['Expected Arrival (HM)'] = pd.to_datetime(arrival_predictions_df['Expected Arrival (HM)'], format='%H:%M')
        arrival_predictions_df['Gap'] = arrival_predictions_df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        arrival_predictions_df['2_Gap'] = (arrival_predictions_df['Gap'] * 2).round(2)
        arrival_predictions_df['Gap_Sq'] = (arrival_predictions_df['Gap'] * arrival_predictions_df['Gap']).round(2)

        cumulative_df = pd.DataFrame(columns=[
            'Line', 'Vehicle ID', 'Stop Point', 'Direction',
            'Expected Arrival (BST)', 'Expected Arrival (HM)',
            'Gap', '2_Gap', 'Gap_Sq'
        ])

        current_hour = datetime.now(pytz.timezone('Europe/London')).hour

        for _, row in arrival_predictions_df.iterrows():
            vehicle_id = row['Vehicle ID']
            if (vehicle_id in cumulative_df['Vehicle ID'].values):
                cumulative_df.loc[(cumulative_df['Vehicle ID'] == vehicle_id), :] = row.values
            else:
                cumulative_df = pd.concat([cumulative_df, row.to_frame().T]).reset_index(drop=True)

        cumulative_df = cumulative_df.sort_values(by='Expected Arrival (BST)', ascending=True)
        cumulative_df['Expected Arrival (BST)'] = pd.to_datetime(cumulative_df['Expected Arrival (BST)'])  # Ensure datetime format
        cumulative_df['Expected Arrival (HM)'] = pd.to_datetime(cumulative_df['Expected Arrival (HM)'], format='%H:%M')

        cumulative_df['Gap'] = cumulative_df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60

        cumulative_df['2_Gap'] = (cumulative_df['Gap'] * 2).round(2)
        cumulative_df['Gap_Sq'] = (cumulative_df['Gap'] * cumulative_df['Gap']).round(2)

        num_buses_observed = cumulative_df[cumulative_df['Expected Arrival (BST)'].dt.hour == current_hour].shape[0]

        return arrival_predictions_df, station_name, cumulative_df, num_buses_observed

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame(), None, pd.DataFrame(), 0

def main():
    line_id = input("Enter Line ID: ")
    stop_point_id = input("Enter Stop Point ID: ")
    direction = input("Enter Direction (inbound/outbound): ")

    while True:
        arrival_predictions_df, station_name, cumulative_df, num_buses_observed = fetch_arrival_predictions(line_id, stop_point_id, direction)

        if not arrival_predictions_df.empty:
            print(f"\nArrival Predictions for stop point {stop_point_id} ({station_name}):")
            print(arrival_predictions_df.to_string(index=False))
            print("\nCumulative DataFrame:")
            print(cumulative_df.to_string(index=False))
            print(f"\nNumber of buses observed in the current hour: {num_buses_observed}")
        else:
            print("No arrival predictions available.")

        print("Refreshing data in 30 seconds...\n")
        time.sleep(30)  # Wait for 30 seconds before fetching data again

if __name__ == "__main__":
    main()


Enter Line ID: 141
Enter Stop Point ID: 490015195M
Enter Direction (inbound/outbound): outbound

Arrival Predictions for stop point 490015195M (City Road / Leonard Street):
Line Vehicle ID Stop Point Direction Expected Arrival (BST) Expected Arrival (HM)  Gap  2_Gap  Gap_Sq
 141    LC67AHE 490015195M  outbound    2024-07-16 17:09:06   1900-01-01 17:09:00  0.0    0.0     0.0
 141    BN14VZK 490015195M  outbound    2024-07-16 17:24:57   1900-01-01 17:24:00 15.0   30.0   225.0

Cumulative DataFrame:
Line Vehicle ID Stop Point Direction Expected Arrival (BST) Expected Arrival (HM)  Gap  2_Gap  Gap_Sq
 141    LC67AHE 490015195M  outbound    2024-07-16 17:09:06   1900-01-01 17:09:00  0.0    0.0     0.0
 141    BN14VZK 490015195M  outbound    2024-07-16 17:24:57   1900-01-01 17:24:00 15.0   30.0   225.0

Number of buses observed in the current hour: 2
Refreshing data in 30 seconds...


Arrival Predictions for stop point 490015195M (City Road / Leonard Street):
Line Vehicle ID Stop Point Direc

KeyboardInterrupt: 

Arrival Predictions for stop point 490015195M (City Road / Leonard Street):
Line Vehicle ID Stop Point Direction Expected Arrival (BST) Expected Arrival (HM)  Gap  2_Gap  Gap_Sq

 141    BJ14KSV 490015195M  outbound    2024-07-16 17:39:45   1900-01-01 17:39:00  2.0    4.0     4.0
 141    BU14EFW 490015195M  outbound    2024-07-16 17:41:52   1900-01-01 17:41:00  2.0    4.0     1.0
  141    BU18EFY 490015195M  outbound    2024-07-16 17:45:07   1900-01-01 17:45:00 11.0   22.0   121.0
 141    LF60EWJ 490015195M  outbound    2024-07-16 17:57:52   1900-01-01 17:57:00  4.0    8.0    16.0